In [1]:
import os
import sys
os.environ['SPARK_HOME']='D:/spark330hdp3sc3'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.9.5-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
spark = SparkSession.builder.appName('FOCMAnalysis') \
.config('spark.warehouse.dir','/apps/hive/warehouse') \
.config('hive.metastore.uris', 'thrift://localhost:9083') \
.enableHiveSupport().getOrCreate()

In [4]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'3.3.0'

In [5]:
from pyspark.sql.functions import *
cm_data_location = 'D:/findataf/cm_sample_data'
fo_data_location = 'D:/findataf/fo_sample_data'

In [40]:
cm_df = spark.read.option(
    "inferSchema", True).option(
    "header", True).csv(cm_data_location)

In [41]:
cm_df.show(2)

+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
|    SYMBOL|SERIES| OPEN| HIGH|  LOW|CLOSE| LAST|PREVCLOSE|TOTTRDQTY|TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|_c13|
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
|182D070324|    TB| 96.0| 96.0| 96.0| 96.0| 96.0|    96.62|      100|   9600.0|11-SEP-2023|          1|IN002023Y243|null|
|182D071223|    TB|97.87|97.87|97.87|97.87|97.87|    97.87|      100|   9787.0|11-SEP-2023|          1|IN002023Y102|null|
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
only showing top 2 rows



In [42]:
cmdf = cm_df.drop("_c13")
print("The cash market data frame schema")
cmdf.printSchema()

The cash market data frame schema
root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)



In [9]:
fo_df = spark.read.option(
    "inferSchema", True).option(
    "header", True).csv(fo_data_location)

In [14]:
fo_df.show(2)

+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|   HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|_c15|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
|    FUTIDX|BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05|46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|null|
|    FUTIDX|BANKNIFTY|31-Aug-2023|      0.0|        XX| 46419.9|46486.1| 45829.7|45986.45| 45986.45|   181369|1254975.82| 1884000|   344535|27-JUL-2023|null|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
only showing top 2 rows



In [15]:
fodf = fo_df.drop("_c15")

In [16]:
def mnameToNo(dt):
    mname = dt[3:6].upper()
    calendar = {"JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
      "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08", "SEP": "09", "OCT": "10",
      "NOV": "11", "DEC": "12"}
    return dt.replace(mname, calendar[mname])

In [17]:
mnameToNo('12-JUN-2023')

'12-06-2023'

In [18]:
udf_mname_to_no = udf(mnameToNo)

In [19]:
fodf.show(2)

+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|   HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX|BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05|46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|
|    FUTIDX|BANKNIFTY|31-Aug-2023|      0.0|        XX| 46419.9|46486.1| 45829.7|45986.45| 45986.45|   181369|1254975.82| 1884000|   344535|27-JUL-2023|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+
only showing top 2 rows



In [20]:
print("verifying that we our udf works and we can plug in to_timestamp")
fodf.select(to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy").alias('rts')).show(2)

verifying that we our udf works and we can plug in to_timestamp
+-------------------+
|                rts|
+-------------------+
|2023-07-27 00:00:00|
|2023-07-27 00:00:00|
+-------------------+
only showing top 2 rows



In [21]:
# register a table against it
fodf.createOrReplaceTempView('fut_data')

In [22]:
print("taking a look at settlement date data")
spark.sql("select * from fut_data where lower(expiry_dt) = lower(timestamp)limit 5").show(3)

taking a look at settlement date data
+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|    HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX|BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05| 46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|
|    FUTIDX|    NIFTY|27-Jul-2023|      0.0|        XX| 19840.2|19872.35| 19600.0| 19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|
|    FUTSTK| AARTIIND|27-Jul-2023|      0.0|        XX|   466.0|   468.5|   456.7|  459.55|   460.25|     5201|  24113.93|  499000| -2971000|27-JUL-2023|
+----------+---------+-----------+----

In [23]:
print("settlement data look using df api")
fodf.where("lower(expiry_dt) = lower(timestamp)").limit(5).show()

settlement data look using df api
+----------+----------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|    SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|    HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+----------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX| BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05| 46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|
|    FUTIDX|     NIFTY|27-Jul-2023|      0.0|        XX| 19840.2|19872.35| 19600.0| 19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|
|    FUTSTK|  AARTIIND|27-Jul-2023|      0.0|        XX|   466.0|   468.5|   456.7|  459.55|   460.25|     5201|  24113.93|  499000| -2971000|27-JUL-2023|
|    FUTSTK|       ABB|27-Jul-2023| 

## F&O Futures Net PL estimation
## Lets say we have the following trades for 3/4 days
### Day  Symbol Long/Short Qty Price  TotalOutlay  OI    CHG_OI
###  1         X                Buy         100   20      2000           100   100
###  2          X               Buy            50   30     3500            150     50
###  3          X                Sell         100    10     2500            50      -100
### Lets say close price on the third day was 12. Then further 12 x 50 needs to be deducted to find out the net settlement amount. 
### So we would have 2500 - 600 - a profit of 1900 made by the sellers

## Lets check a profitable scenario for the buyers
### Day  Symbol Long/Short Qty Price  TotalOutlay  OI    CHG_OI
###  1         X                Buy         100   20      2000           100   100
###  2          X               Buy            50   30     3500            150     50
###  3          X                Sell         100    40     -500            50      -100
### Lets say close price on the third day was 42. Then further 42 x 50 needs to be deducted to find out the net settlement amount. 
### So we would have -500-2100 - a profit of 2600 made by the buyers



In [26]:
spark.udf.register("udf_mname_to_no", mnameToNo)

print("Using sql to show overall profit loss status for f&o market")
spark.sql("""
select timestamp, symbol,expiry_dt,chg_in_oi,close,open_int,
sum(chg_in_oi)
over (partition by symbol,expiry_dt, option_typ order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")) as cum_chg_oi,
sum(chg_in_oi*close)
over ( partition by symbol,expiry_dt, option_typ order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy"))
as b_s_pl_status
from fut_data
where instrument like 'FUT%' and symbol = 'INFY' and expiry_dt = '28-Sep-2023'
order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")
""").show(100, False)

Using sql to show overall profit loss status for f&o market
+-----------+------+-----------+---------+-------+--------+----------+--------------+
|timestamp  |symbol|expiry_dt  |chg_in_oi|close  |open_int|cum_chg_oi|b_s_pl_status |
+-----------+------+-----------+---------+-------+--------+----------+--------------+
|30-JUN-2023|INFY  |28-Sep-2023|44000    |1364.55|44000   |44000     |6.00402E7     |
|03-JUL-2023|INFY  |28-Sep-2023|57200    |1359.65|101200  |101200    |1.3781218E8   |
|04-JUL-2023|INFY  |28-Sep-2023|23200    |1369.85|124400  |124400    |1.695927E8    |
|05-JUL-2023|INFY  |28-Sep-2023|13600    |1370.75|138000  |138000    |1.882349E8    |
|06-JUL-2023|INFY  |28-Sep-2023|21600    |1371.05|159600  |159600    |2.1784958E8   |
|07-JUL-2023|INFY  |28-Sep-2023|50800    |1352.5 |210400  |210400    |2.8655658E8   |
|10-JUL-2023|INFY  |28-Sep-2023|6000     |1353.35|216400  |216400    |2.9467668E8   |
|11-JUL-2023|INFY  |28-Sep-2023|-4400    |1370.3 |212000  |212000    |2.8864736E

In [27]:
# create a view with the proper timestamp column - tsp -timestamp proper
fodfvw = fodf.withColumn("tsp", to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy"))

In [28]:
from pyspark.sql import Window
window_spec = Window.partitionBy("symbol", "expiry_dt", "instrument").orderBy("tsp")
#then define the aggregation functions we are interested in over it
sumChgoi = sum(col("chg_in_oi")).over(window_spec)
plStatus = sum(col('chg_in_oi') * col('close')).over(window_spec)

In [29]:
# and plug in the regular dataframe api
print("window functions executed using df api")
fodfvw.select( "tsp","symbol", 
    "expiry_dt", "chg_in_oi", "close", "open_int",
    sumChgoi.alias("cum_chg_oi"), plStatus.alias("b_s_pl_status")).filter(
    "symbol = 'INFY' and instrument like 'FUT%' and expiry_dt = '28-Sep-2023'").orderBy("tsp").show(100, False)

window functions executed using df api
+-------------------+------+-----------+---------+-------+--------+----------+--------------+
|tsp                |symbol|expiry_dt  |chg_in_oi|close  |open_int|cum_chg_oi|b_s_pl_status |
+-------------------+------+-----------+---------+-------+--------+----------+--------------+
|2023-06-30 00:00:00|INFY  |28-Sep-2023|44000    |1364.55|44000   |44000     |6.00402E7     |
|2023-07-03 00:00:00|INFY  |28-Sep-2023|57200    |1359.65|101200  |101200    |1.3781218E8   |
|2023-07-04 00:00:00|INFY  |28-Sep-2023|23200    |1369.85|124400  |124400    |1.695927E8    |
|2023-07-05 00:00:00|INFY  |28-Sep-2023|13600    |1370.75|138000  |138000    |1.882349E8    |
|2023-07-06 00:00:00|INFY  |28-Sep-2023|21600    |1371.05|159600  |159600    |2.1784958E8   |
|2023-07-07 00:00:00|INFY  |28-Sep-2023|50800    |1352.5 |210400  |210400    |2.8655658E8   |
|2023-07-10 00:00:00|INFY  |28-Sep-2023|6000     |1353.35|216400  |216400    |2.9467668E8   |
|2023-07-11 00:00:00|

In [30]:
# here we check the status on the expiry dates for the pl situation
print("status on the expiry dates for the pl situation")
spark.sql("""
select f.* from
(
select timestamp, symbol,expiry_dt,chg_in_oi,close,open_int,
sum(chg_in_oi) over (partition by symbol,expiry_dt, option_typ 
order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")) as cum_chg_oi,
sum(chg_in_oi*close) over ( partition by symbol,expiry_dt, option_typ 
order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy"))
as b_s_pl_status
from fut_data
where instrument like 'FUT%'
) f
where lower(f.expiry_dt) = lower(f.timestamp) and symbol = 'INFY'
order by to_timestamp(udf_mname_to_no(f.timestamp), "dd-MM-yyyy")
""").show(50, False)

status on the expiry dates for the pl situation
+-----------+------+-----------+---------+-------+--------+----------+--------------+
|timestamp  |symbol|expiry_dt  |chg_in_oi|close  |open_int|cum_chg_oi|b_s_pl_status |
+-----------+------+-----------+---------+-------+--------+----------+--------------+
|28-JUN-2023|INFY  |28-Jun-2023|-10016000|1289.9 |1361600 |-10016000 |-1.29196384E10|
|27-JUL-2023|INFY  |27-Jul-2023|-4154800 |1353.8 |1466800 |149600    |-2.25148718E9 |
|31-AUG-2023|INFY  |31-Aug-2023|-5136400 |1436.05|6224000 |6010400   |7.00869112E9  |
|28-SEP-2023|INFY  |28-Sep-2023|-4058000 |1442.95|2902800 |2902800   |3.45143592E9  |
+-----------+------+-----------+---------+-------+--------+----------+--------------+



In [43]:
# create a proper timestamp column for the cash market data
cmdfvw = cmdf.withColumn(
    "tsp",
    to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy"))

cmdfvw.createOrReplaceTempView("cmdata")

print("verifying the extra column we added as a proper timestamp")
cmdfvw.show(5, False)

verifying the extra column we added as a proper timestamp
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+-------------------+
|SYMBOL    |SERIES|OPEN |HIGH |LOW  |CLOSE|LAST |PREVCLOSE|TOTTRDQTY|TOTTRDVAL|TIMESTAMP  |TOTALTRADES|ISIN        |tsp                |
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+-------------------+
|182D070324|TB    |96.0 |96.0 |96.0 |96.0 |96.0 |96.62    |100      |9600.0   |11-SEP-2023|1          |IN002023Y243|2023-09-11 00:00:00|
|182D071223|TB    |97.87|97.87|97.87|97.87|97.87|97.87    |100      |9787.0   |11-SEP-2023|1          |IN002023Y102|2023-09-11 00:00:00|
|182D121023|TB    |99.23|99.25|99.23|99.25|99.25|99.23    |1700     |168721.0 |11-SEP-2023|2          |IN002023Y029|2023-09-11 00:00:00|
|182D161123|TB    |98.55|98.55|98.55|98.55|98.55|98.45    |300      |29565.0  |11-SEP-2023|1          |IN002023Y078|2023

In [45]:
# use sql to carry out window functions moving average using rows preceding
print("using sql to carry out window functions moving average using rows preceding")
spark.sql("""
select symbol, timestamp,close,
round(avg(close) over(partition by symbol order by tsp rows 5 preceding )) as mv5avg,
round(avg(close) over(partition by symbol order by tsp rows 20 preceding )) as mv20avg
from cmdata
where symbol = 'INFY'
order by tsp
""").show()

using sql to carry out window functions moving average using rows preceding
+------+-----------+-------+------+-------+
|symbol|  timestamp|  close|mv5avg|mv20avg|
+------+-----------+-------+------+-------+
|  INFY|01-JUN-2023| 1319.5|1320.0| 1320.0|
|  INFY|02-JUN-2023| 1299.0|1309.0| 1309.0|
|  INFY|05-JUN-2023| 1304.5|1308.0| 1308.0|
|  INFY|06-JUN-2023| 1279.0|1301.0| 1301.0|
|  INFY|07-JUN-2023|1288.95|1298.0| 1298.0|
|  INFY|08-JUN-2023| 1282.8|1296.0| 1296.0|
|  INFY|09-JUN-2023|1265.95|1287.0| 1291.0|
|  INFY|12-JUN-2023|1291.95|1286.0| 1291.0|
|  INFY|13-JUN-2023|1304.85|1286.0| 1293.0|
|  INFY|14-JUN-2023| 1300.4|1289.0| 1294.0|
|  INFY|15-JUN-2023| 1286.8|1289.0| 1293.0|
|  INFY|16-JUN-2023|1291.65|1290.0| 1293.0|
|  INFY|19-JUN-2023| 1294.0|1295.0| 1293.0|
|  INFY|20-JUN-2023| 1303.4|1297.0| 1294.0|
|  INFY|21-JUN-2023|1299.35|1296.0| 1294.0|
|  INFY|22-JUN-2023|1281.55|1293.0| 1293.0|
|  INFY|23-JUN-2023| 1265.0|1289.0| 1292.0|
|  INFY|26-JUN-2023|1270.35|1286.0| 1290.0|


In [50]:
# using dataframe api to calculate moving average
print("using dataframe api to calculate moving average")
mvngAvgSpec = avg("close").over(
    Window.partitionBy("symbol").orderBy("tsp").rowsBetween(-3, 0))
 
cmdfvw.select("symbol", "timestamp", "tsp", "close",
    round(mvngAvgSpec).alias("mvng_avg_5")).filter(
    "symbol = 'INFY'").orderBy(
    "tsp").show()

using dataframe api to calculate moving average
+------+-----------+-------------------+-------+----------+
|symbol|  timestamp|                tsp|  close|mvng_avg_5|
+------+-----------+-------------------+-------+----------+
|  INFY|01-JUN-2023|2023-06-01 00:00:00| 1319.5|    1320.0|
|  INFY|02-JUN-2023|2023-06-02 00:00:00| 1299.0|    1309.0|
|  INFY|05-JUN-2023|2023-06-05 00:00:00| 1304.5|    1308.0|
|  INFY|06-JUN-2023|2023-06-06 00:00:00| 1279.0|    1301.0|
|  INFY|07-JUN-2023|2023-06-07 00:00:00|1288.95|    1293.0|
|  INFY|08-JUN-2023|2023-06-08 00:00:00| 1282.8|    1289.0|
|  INFY|09-JUN-2023|2023-06-09 00:00:00|1265.95|    1279.0|
|  INFY|12-JUN-2023|2023-06-12 00:00:00|1291.95|    1282.0|
|  INFY|13-JUN-2023|2023-06-13 00:00:00|1304.85|    1286.0|
|  INFY|14-JUN-2023|2023-06-14 00:00:00| 1300.4|    1291.0|
|  INFY|15-JUN-2023|2023-06-15 00:00:00| 1286.8|    1296.0|
|  INFY|16-JUN-2023|2023-06-16 00:00:00|1291.65|    1296.0|
|  INFY|19-JUN-2023|2023-06-19 00:00:00| 1294.0|    

In [51]:
# exercise - add on 20 days moving average

# using row number

print("using row number with sql")

spark.sql("""
select symbol, timestamp, close,
row_number() over(partition by symbol order by tsp) as rno
from cmdata
where symbol in ('INFY', 'TCS')""").show()

using row number with sql
+------+-----------+-------+---+
|symbol|  timestamp|  close|rno|
+------+-----------+-------+---+
|  INFY|01-JUN-2023| 1319.5|  1|
|  INFY|02-JUN-2023| 1299.0|  2|
|  INFY|05-JUN-2023| 1304.5|  3|
|  INFY|06-JUN-2023| 1279.0|  4|
|  INFY|07-JUN-2023|1288.95|  5|
|  INFY|08-JUN-2023| 1282.8|  6|
|  INFY|09-JUN-2023|1265.95|  7|
|  INFY|12-JUN-2023|1291.95|  8|
|  INFY|13-JUN-2023|1304.85|  9|
|  INFY|14-JUN-2023| 1300.4| 10|
|  INFY|15-JUN-2023| 1286.8| 11|
|  INFY|16-JUN-2023|1291.65| 12|
|  INFY|19-JUN-2023| 1294.0| 13|
|  INFY|20-JUN-2023| 1303.4| 14|
|  INFY|21-JUN-2023|1299.35| 15|
|  INFY|22-JUN-2023|1281.55| 16|
|  INFY|23-JUN-2023| 1265.0| 17|
|  INFY|26-JUN-2023|1270.35| 18|
|  INFY|27-JUN-2023|1279.15| 19|
|  INFY|28-JUN-2023|1293.35| 20|
+------+-----------+-------+---+
only showing top 20 rows



In [52]:
# dafaframe api using row number
print("dafaframe api showing row number")
rnospec = row_number().over(
    Window.partitionBy("symbol").orderBy(
    "tsp"))

cmdfvw.select("symbol", "timestamp", "close",
    rnospec.alias("rno")).filter(
    "symbol in ('INFY','TCS')").show()

dafaframe api showing row number
+------+-----------+-------+---+
|symbol|  timestamp|  close|rno|
+------+-----------+-------+---+
|  INFY|01-JUN-2023| 1319.5|  1|
|  INFY|02-JUN-2023| 1299.0|  2|
|  INFY|05-JUN-2023| 1304.5|  3|
|  INFY|06-JUN-2023| 1279.0|  4|
|  INFY|07-JUN-2023|1288.95|  5|
|  INFY|08-JUN-2023| 1282.8|  6|
|  INFY|09-JUN-2023|1265.95|  7|
|  INFY|12-JUN-2023|1291.95|  8|
|  INFY|13-JUN-2023|1304.85|  9|
|  INFY|14-JUN-2023| 1300.4| 10|
|  INFY|15-JUN-2023| 1286.8| 11|
|  INFY|16-JUN-2023|1291.65| 12|
|  INFY|19-JUN-2023| 1294.0| 13|
|  INFY|20-JUN-2023| 1303.4| 14|
|  INFY|21-JUN-2023|1299.35| 15|
|  INFY|22-JUN-2023|1281.55| 16|
|  INFY|23-JUN-2023| 1265.0| 17|
|  INFY|26-JUN-2023|1270.35| 18|
|  INFY|27-JUN-2023|1279.15| 19|
|  INFY|28-JUN-2023|1293.35| 20|
+------+-----------+-------+---+
only showing top 20 rows



In [53]:
# creating a view that we will use to cube by year, month, day
cmdfymdvw = cmdfvw.withColumn(
    "yr", year("tsp")).withColumn(
    "mnth", month("tsp")).withColumn(
    "dy", dayofmonth("tsp"))

In [54]:
cmdfymdvw.createOrReplaceTempView("cb_tbl")

In [55]:
# grouping sets using sql
print("grouping sets using sql")
spark.sql("""
select symbol, yr, mnth, dy , avg(close) as avgcls, avg(tottrdqty) as avgqty, avg(tottrdval) as avgval from cb_tbl
where symbol in ('INFY', 'TCS')
group by symbol, yr, mnth, dy
grouping sets((yr), (yr, mnth))
""").show()

grouping sets using sql
+------+----+----+----+------------------+------------------+--------------------+
|symbol|  yr|mnth|  dy|            avgcls|            avgqty|              avgval|
+------+----+----+----+------------------+------------------+--------------------+
|  null|2023|   8|null| 2386.785555555555|3204309.1555555556| 6.087381503196668E9|
|  null|2023|null|null|2381.5233918128656|4141724.0994152045| 7.600556120269882E9|
|  null|2023|   9|null|2502.8149999999996|       3499600.125| 7.146561955257498E9|
|  null|2023|   7|null|2375.5772727272724|5918001.0227272725|1.030950020456363...|
|  null|2023|   6|null|2266.5988095238095|3896782.3571428573| 6.816248612171429E9|
+------+----+----+----+------------------+------------------+--------------------+



In [56]:
#cube using sql
print("cube using sql")
spark.sql("""
select symbol,yr,mnth,dy,
avg(close) as avgcls,
avg(tottrdqty) as avgqty,
avg(tottrdval) as avgval
from cb_tbl
group by symbol,yr,mnth,dy with cube
order by yr ,mnth""").show()

cube using sql
+----------+----+----+----+------------------+--------------------+--------------------+
|    symbol|  yr|mnth|  dy|            avgcls|              avgqty|              avgval|
+----------+----+----+----+------------------+--------------------+--------------------+
|     MOXSH|null|null|  11|            138.45|              4800.0|            651320.0|
|LTGILTBEES|null|null|   4|             24.22|   392429.3333333333|          9505568.12|
| NILAINFRA|null|null|  11|              6.45|   4320781.666666667|3.4301667833333336E7|
| AXISILVER|null|null|  11|             72.27|  17334.333333333332|  1251319.7833333334|
|  PROZONER|null|null|null| 29.20384615384615|  1234307.8205128205| 3.744807216282052E7|
|     CLSEL|null|null|  11|187.66666666666666|  183064.33333333334|        3.40481601E7|
|    RECLTD|null|null|null|1045.1475206611572|  3490264.7066115704| 7.615634056035123E8|
|     GLOBE|null|null|null|3.2212121212121216|   1081196.803030303|  4639345.1462121215|
|    S

In [57]:
# cube using dafaframe api
print("cube using dafaframe api")
cmdfymdvw.cube("symbol", "yr", "mnth", "dy").agg(
      avg("close").alias("avgcls"),
      avg("tottrdqty").alias("avgqty"),
      avg("tottrdval").alias("tottrdval")).show()

cube using dafaframe api
+----------+----+----+---+-------+--------+-------------+
|    symbol|  yr|mnth| dy| avgcls|  avgqty|    tottrdval|
+----------+----+----+---+-------+--------+-------------+
|   ABSLAMC|2023|   9| 11|  425.7|172048.0| 7.32939608E7|
|       ACI|2023|   9| 11| 641.35|381754.0|2.463451069E8|
|  AVONMORE|2023|   9| 11|  76.35| 16633.0|    1277943.6|
|DLINKINDIA|2023|   9| 11| 343.85|191756.0| 6.64023869E7|
|      EMIL|2023|   9| 11|  139.6|378676.0| 5.28313908E7|
|JAYSREETEA|2023|   9| 11|   99.6| 71763.0|   7183523.95|
|      NETF|2023|   9| 11| 210.16|  2468.0|    517999.34|
|   REVATHI|2023|   9| 11|1805.05|  8787.0| 1.57519437E7|
|SHARIABEES|2023|   9| 11| 445.17|  3914.0|   1737284.98|
|       SHK|2023|   9| 11|  163.8|360439.0|5.953054385E7|
|     SPARC|2023|   9| 11| 253.45|478242.0|1.219817051E8|
|SUPREMEINF|2023|   9| 11|   27.8| 35033.0|     973917.4|
| AXISNIFTY|2023|   9|  4| 209.13|  3081.0|    642823.27|
|   KSOLVES|2023|   9|  4| 1100.6| 53228.0| 5.8

In [58]:
# cube - see the overall aggregates
print("cube - see the overall aggregates")
cmdfymdvw.cube("symbol", "yr", "mnth", "dy"). agg(
      avg("close").alias("avgcls"),
      avg("tottrdqty").alias("avgqty"),
      avg("tottrdval").alias("tottrdval")).filter(
    isnull("symbol")).show()

cube - see the overall aggregates
+------+----+----+----+-----------------+------------------+--------------------+
|symbol|  yr|mnth|  dy|           avgcls|            avgqty|           tottrdval|
+------+----+----+----+-----------------+------------------+--------------------+
|  null|2023|null|   4| 836.448175982883|2003058.8411339931| 3.159584898047006E8|
|  null|2023|   9|   4|845.9586381475672| 2618987.836342229|3.5694157723716694E8|
|  null|2023|null|  28|831.9763298139766|1465931.7235796882| 3.054817723830453E8|
|  null|null|   9|  26|850.4475326215882|1446213.9023329378| 2.461903171638947E8|
|  null|2023|   9|  12|847.2899011857703| 2885374.771146245|4.2797661508579046E8|
|  null|null|   9|   5|857.8224733727817|2257206.8915187377| 3.500626328693299E8|
|  null|2023|   9|null|848.3942434992844|2039790.4679720944| 3.311413305395452E8|
|  null|null|   9|  18|834.6526037588089| 2554883.552466719| 3.080921622156301E8|
|  null|2023|   9|   7|856.9022402854874|1996773.2549563837| 3.2

In [60]:
# rolling using sql
print("rolling using sql")
spark.sql("""
select symbol,yr,mnth,dy,
avg(close) as avgcls, avg(tottrdqty) as avgqty,
avg(tottrdval) as avgval from cb_tbl
where symbol = 'INFY' and mnth = 6
group by symbol,yr,mnth,dy with rollup
order by yr ,mnth, dy""").show()

rolling using sql
+------+----+----+----+-----------------+-----------+-------------------+
|symbol|  yr|mnth|  dy|           avgcls|     avgqty|             avgval|
+------+----+----+----+-----------------+-----------+-------------------+
|  INFY|null|null|null|1292.238095238095|  5966804.0|7.718180427314284E9|
|  null|null|null|null|1292.238095238095|  5966804.0|7.718180427314284E9|
|  INFY|2023|null|null|1292.238095238095|  5966804.0|7.718180427314284E9|
|  INFY|2023|   6|null|1292.238095238095|  5966804.0|7.718180427314284E9|
|  INFY|2023|   6|   1|           1319.5|  4277841.0|    5.65917688955E9|
|  INFY|2023|   6|   2|           1299.0|1.0909311E7|  1.421040134745E10|
|  INFY|2023|   6|   5|           1304.5|  5002553.0|    6.53094046645E9|
|  INFY|2023|   6|   6|           1279.0|  1.00081E7|   1.27997830858E10|
|  INFY|2023|   6|   7|          1288.95|  5397578.0|     6.9537037304E9|
|  INFY|2023|   6|   8|           1282.8|  5034942.0|     6.4769757639E9|
|  INFY|2023|   6|  